In [3]:
import pandas as pd
import sys
sys.path.insert(0,"./..")
from src.PassivePy import PassivePyAnalyzer
from src import analysis_passive

#analyzer = PassivePyAnalyzer(spacy_model= "fr_core_news_lg")


import spacy
from spacy.matcher import Matcher
nlp = spacy.load("fr_core_news_lg")

In [35]:
with open('/home/robin/Code_repo/app_passive_detection/data/intransif_verbs.txt') as f : 
   lines =  f.readlines()

In [41]:
l = [elt.replace("\n","").lower().strip() for elt in  open('/home/robin/Code_repo/app_passive_detection/data/intransif_verbs.txt').readlines()[1:]]

In [43]:
len(l)

2299

In [7]:
import spacy_stanza

/home/robin/env/app_passive/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
nlp = spacy_stanza.load_pipeline("fr" )

2022-12-07 10:10:53.525 INFO    stanza: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-07 10:10:55.896 INFO    stanza: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2022-12-07 10:10:55.898 INFO    stanza: Use device: gpu
2022-12-07 10:10:55.899 INFO    stanza: Loading: tokenize
2022-12-07 10:10:55.915 INFO    stanza: Loading: mwt
2022-12-07 10:10:55.923 INFO    stanza: Loading: pos
2022-12-07 10:10:56.363 INFO    stanza: Loading: lemma
2022-12-07 10:10:56.427 INFO    stanza: Loading: depparse
2022-12-07 10:10:56.734 INFO    stanza: Loading: ner
2022-12-07 10:10:58.495 INFO    stanza: Done loading processors!


In [33]:
doc  = nlp("Spiderman a été vu au cinéma. La pomme a été posée sur la table ! J'ai fais à manger des pattes à mes amis.")
totalVerbs = [token for token in doc if (token.pos_ == "VERB")]
totalVerbs

/home/robin/env/app_passive/lib/python3.8/site-packages/spacy/language.py:1014: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  doc = self._ensure_doc(text)
/home/robin/env/app_passive/lib/python3.8/site-packages/spacy/language.py:1014: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['Spiderman', 'a', 'été', 'vu', 'à', 'le', 'cinéma', '.', 'La', 'pomme', 'a', 'été', 'posée', 'sur', 'la', 'table', '!', "J'", 'ai', 'fais', 'à', 'manger', 'des', 'pattes', 'à', 'mes', 'amis', '.']
Entities: [('Spiderman', 'MISC', 0, 9), ('La pomme', 'MISC', 30, 38)]
  doc = self._ensure_doc(text)


[vu, posée, fais, manger]

In [34]:

for verb in totalVerbs:
    # direct object check
    directObject = False
    # indirect object check
    indirectObject = False
    
    
    print(verb)
    # looks through each verb's child ie dependents
    for item in verb.children:
    
        print(item.dep_)
        # sets indirectObject to true if indirect object is found
        if(item.dep_ == "iobj" or item.dep_ == "pobj"):
            indirectObject = True
        # sets directObject to true if direct object is found
        if (item.dep_ == "dobj" or item.dep_ == "dative"):
            directObject = True
            

vu
nsubj:pass
aux:tense
aux:pass
obl:arg
punct
posée
nsubj:pass
aux:tense
aux:pass
obl:mod
punct
fais
nsubj
aux:tense
xcomp
punct
manger
mark
obj
obl:mod


In [7]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("fr_core_news_lg")
matcher = Matcher(nlp.vocab)
r = [
        {"TAG":"ADJ", "TEXT" : {"REGEX": "\b(\w*(ible|able))\b"}}, # adjectif se finnissant par ible ou able
    ]
matcher.add("test", [r], greedy='LONGEST')

In [18]:
matcher = Matcher(nlp.vocab)
r = [
        {"TAG":"ADJ","TEXT" : {"REGEX": r"\b(\w*(ible|able))\b"}}, # adjectif se finnissant par ible ou able
    ]
matcher.add("test", [r], greedy='LONGEST')

doc = nlp("cette ile est visible depuis l'ocean")
for match in matcher(doc):
    print(match)

(1618900948208871284, 3, 4)


In [8]:
data = pd.DataFrame()
data['text'] =  ["elle a été déterminée et formée ! ",
"accusé par la police, il fut condamné a 5 ans de prison ferme"
,"ma mère a été élevée par une euh une une vieille dame euh à euh dans l'Allier"
,"puis on a été soutenu par un monsieur aussi c'est monsieur NPERS…",
"la la République du Centre a été rachetée euh par un journal euh je voudrais pas dire de bêtises je crois que c'est Clermont-Ferrand Clermont enfin…",
"alors il y a des écoles qui sont faites exclusivement par des religieuses ou certains par des prêtres mais alors maintenant ils ont beaucoup d'institutrices euh civiles qui sont dedans",
"donc il est il est payé quoi bah c'est super",
"à à mon avis ça a déjà été fait mais au niveau euh professionnel au niveau du travail je crois que ça se développe ouais",
"quand tu habites dans le centre d'Orléans euh c'est quand même réservé à un public qui a … un minimum d'argent",
"parce que même avant on a été coincé euh pendant pas mal d'années euh sans voiture et on travaillait pas","tu es à peine payé et tout quoi",
"tout à l'heure j'avais été inter- la dernière fois que j'avais été interviewée c'était pour la la construction de la nouvelle fac",
"donc vous êtes  envoyée ponctuellement sur des sur des sur des sites sites",
"d'accord je devais pas rester à Orléans je devais juste être formée pendant un an et partir sur Lille",
"je trouve que pour le moment euh c'est pas encore trop bien aménagé euh pour les vélos hein vélos",
"Les adultes quand tu habites dans le centre d'Orléans euh c'est quand même réservé à un public qui a hm ouais ouais bien sûr ouais un minimum d'argent"
"des  des échanges qui qui se font pas par téléphone"]

In [3]:
data, cols = analysis_passive.load_passive_features(data, analyzer)

Detecting Sentences...
Starting to find passives...


In [2]:
text = "J'ai été attaqué par des loups !"
samples = analyzer.print_matches(text)
doc = analyzer.prepare_visualisation(text)



été attaqué
rule:  passif_tronqué
été attaqué par
rule:  passif_canonique
attaqué par des loups
rule:  passif_impersonel


In [1]:
from spacy_streamlit import visualize_spans
matches, doc = analyzer.prepare_visualisation(text)
visualize_spans(doc, spans_key="passive", show_table =  False, title = "")

NameError: name 'analyzer' is not defined

In [9]:
import spacy
nlp = spacy.load("fr_core_news_lg")
doc = nlp("je suis un exemple de text")

In [48]:
doc = nlp("elle a été déterminée et formée ! La pomme a été mangé par le chat")
ids = [(elt[1],elt[2]) for elt in passivepy.matcher(doc)]
samples = []

color_dict = {
    "passif_tronqué": "#8ef",
    "passive_rule_2" : "#faa",
    "passif_sequencé" : "afa",
    "passif_impersonel":"#fea",
    "passive_rule_5":"#faa",
    "passif_verbale":"#000080",
    "passif_factif" : "#00FF00",
    "passif_adjectif":"#808080"   
}

for token in doc :
    for id_, s,e  in passivepy.matcher(doc) :
        if not s<token.i<e :
            samples.append((token.text))
        else :
            rule_name = "test"#nlp.vocab.strings[id_]
            rule_color = "C"#color_dict[rule_name]
            samples.append((token.text,rule_name, rule_color))


In [50]:
doc.text

'elle a été déterminée et formée ! La pomme a été mangé par le chat'